In [2]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, roc_curve, auc, roc_auc_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.dummy import DummyClassifier
# import train_test_split from sklearn.model_selection
from sklearn.model_selection import train_test_split, GridSearchCV


In [35]:
input_data_qm = pd.read_csv("aki_data/test_qm.csv")
design_matrix = pd.read_csv("aki_data/design_matrix.tsv", sep="\t")

In [36]:
input_data_qm.head()

,Protein,TM_P1911_190,TM_P1911_191,TM_P1911_192,TM_P1911_193,TM_P1911_194,TM_P1911_196,TM_P1911_197,TM_M2012_010,TM_M2012_011,...,TM_M2012_190,TM_M2012_191,TM_M2012_192,TM_M2012_196,TM_M2012_197,TM_M2012_198,TM_M2012_199,TM_M2012_200,TM_M2012_202,TM_M2012_203
0,P08603,22.381866,22.773908,22.732549,22.960530,22.906198,23.167862,23.122564,23.110142,23.179716,...,23.416677,23.498007,23.459972,23.403313,23.454894,23.602666,23.682634,23.665858,24.015710,23.655648
1,P02671,25.349974,25.431340,25.459891,25.275259,25.592789,24.829806,24.208987,23.984077,26.075865,...,24.984516,25.023149,24.971465,23.369445,24.604836,24.623221,24.787905,25.095571,25.103341,24.914344
2,P01042,22.061788,21.872170,21.966596,22.256140,22.505168,22.993978,23.277504,22.963205,22.767097,...,22.953879,23.089170,23.018547,23.280626,23.503529,23.471356,23.471414,23.193750,24.101306,23.486766
3,P00450,22.647246,23.193086,23.332780,23.206429,22.959381,23.008403,22.770807,22.971128,23.373016,...,23.788756,23.932623,23.904721,23.273831,23.462794,23.783564,23.968122,23.956618,23.989086,23.834912
4,P05156,21.301448,21.435684,21.304184,21.459141,21.532018,22.006447,21.968122,21.688934,21.372610,...,21.850530,21.883567,21.936084,21.778412,22.051000,22.187546,21.965964,21.820840,22.373783,22.076671


In [52]:
# split data into training and test set
# every column that contains M2012 is test set
input_data_preprocessed = input_data_qm.fillna(0)
input_data = input_data_preprocessed.drop(['Protein'], axis=1)
X_test = input_data.loc[:, ~input_data.columns.str.contains('M2012')].transpose()
X_train = input_data.loc[:, input_data.columns.str.contains('M2012')].transpose()
# split design matrix into training and test set
# first row contains patient id, split by M2012, then remove first row
y_test = design_matrix['group'][~design_matrix['sample'].str.contains('M2012')]
y_train = design_matrix['group'][design_matrix['sample'].str.contains('M2012')]
# change labels to 0 and 1 from 1 and 2
y_test = y_test.replace(1, 0)
y_test = y_test.replace(2, 1)
y_train = y_train.replace(1, 0)
y_train = y_train.replace(2, 1)

print(X_test.shape)
print(X_train.shape)
print(y_test.shape)
print(y_train.shape)

(56, 554)
(141, 554)
(56,)
(141,)


In [33]:
input_data_preprocessed.head()

,Protein,TM_P1911_190,TM_P1911_191,TM_P1911_192,TM_P1911_193,TM_P1911_194,TM_P1911_196,TM_P1911_197,TM_M2012_010,TM_M2012_011,...,TM_M2012_190,TM_M2012_191,TM_M2012_192,TM_M2012_196,TM_M2012_197,TM_M2012_198,TM_M2012_199,TM_M2012_200,TM_M2012_202,TM_M2012_203
0,P08603,22.381866,22.773908,22.732549,22.960530,22.906198,23.167862,23.122564,23.110142,23.179716,...,23.416677,23.498007,23.459972,23.403313,23.454894,23.602666,23.682634,23.665858,24.015710,23.655648
1,P02671,25.349974,25.431340,25.459891,25.275259,25.592789,24.829806,24.208987,23.984077,26.075865,...,24.984516,25.023149,24.971465,23.369445,24.604836,24.623221,24.787905,25.095571,25.103341,24.914344
2,P01042,22.061788,21.872170,21.966596,22.256140,22.505168,22.993978,23.277504,22.963205,22.767097,...,22.953879,23.089170,23.018547,23.280626,23.503529,23.471356,23.471414,23.193750,24.101306,23.486766
3,P00450,22.647246,23.193086,23.332780,23.206429,22.959381,23.008403,22.770807,22.971128,23.373016,...,23.788756,23.932623,23.904721,23.273831,23.462794,23.783564,23.968122,23.956618,23.989086,23.834912
4,P05156,21.301448,21.435684,21.304184,21.459141,21.532018,22.006447,21.968122,21.688934,21.372610,...,21.850530,21.883567,21.936084,21.778412,22.051000,22.187546,21.965964,21.820840,22.373783,22.076671


In [54]:
design_matrix[design_matrix['sample'].str.contains('M2012')]

,sample,group
7,TM_M2012_010,1
8,TM_M2012_011,1
9,TM_M2012_012,1
10,TM_M2012_013,1
11,TM_M2012_014,1
...,...,...
192,TM_M2012_198,2
193,TM_M2012_199,2
194,TM_M2012_200,2
195,TM_M2012_202,2


In [97]:
models = [DummyClassifier(random_state = SEED),
          SVC(random_state = SEED, probability = True),
          LogisticRegression(random_state = SEED, max_iter = 1000),
          RandomForestClassifier(random_state = SEED),
          GradientBoostingClassifier(random_state = SEED)]

grid = {"Dummy": {"strategy": ["most_frequent"]},
        "SVC": {"C": [0.1, 1, 2], "kernel": ["linear", "rbf", "poly"]},
        "LR": {"C": [0.01, 0.1, 1], "penalty": ["l1", "l2", "elasticnet"]},
        "RF": {"n_estimators": [50, 100, 150]},
        "GB": {"learning_rate": [0.05, 0.001, 0.0005], "n_estimators": [50, 100, 150]}
}

# train models
best_scores = {}
best_params = {}

for model_name, names in zip(models, grid.keys()):
    print("Training model %s" % names)
    model = GridSearchCV(model_name, grid[names], cv = 10, scoring = "f1_macro", n_jobs=8)
    model.fit(X_train, y_train)
    best_scores[names] = model.best_score_
    best_params[names] = model.best_params_

    # print best parameters and best score
    print("Best parameters: %s" % model.best_params_)
    print("Best score: %f" % model.best_score_)

best_models = pd.DataFrame({"Model": list(best_scores.keys()),
                                "Best score": list(best_scores.values()),
                                "Best params": list(best_params.values())})

best_models.sort_values("Best score", ascending=False, inplace=True)




Training model Dummy
Best parameters: {'strategy': 'most_frequent'}
Best score: 0.364773
Training model SVC
Best parameters: {'C': 0.1, 'kernel': 'linear'}
Best score: 0.871513
Training model LR


/Users/Yves.Gorgen/anaconda3/envs/adlg/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
60 fits failed out of a total of 90.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/Yves.Gorgen/anaconda3/envs/adlg/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/Yves.Gorgen/anaconda3/envs/adlg/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/Users/Yves.Gorgen/anaconda3/envs/adlg/lib/python3.9/site-pa

Best parameters: {'C': 1, 'penalty': 'l2'}
Best score: 0.861221
Training model RF
Best parameters: {'n_estimators': 150}
Best score: 0.824277
Training model GB
Best parameters: {'learning_rate': 0.001, 'n_estimators': 150}
Best score: 0.709632


In [98]:
# Using each model with best parameters, predict test set
# calculate AUC, ROC curve, confusion matrix

def predict_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    cm = confusion_matrix(y_test, y_pred)
    return y_pred, y_pred_proba, cm


def plot_confusion_matrix(cm):
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion Matrix")
    plt.colorbar()
    plt.xticks([0,1], ["No AKI", "AKI"])
    plt.yticks([0,1], ["No AKI", "AKI"])
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.show()

def print_metrics(cm, y_pred, y_pred_proba):
    recall_pheno1 = cm[1,1] / (cm[1,1] + cm[1,0] + 1e-10)
    recall_pheno0 = cm[0,0] / (cm[0,0] + cm[0,1] + 1e-10)
    precision_pheno1 = cm[1,1] / (cm[1,1] + cm[0,1] + 1e-10)
    precision_pheno0 = cm[0,0] / (cm[0,0] + cm[1,0] + 1e-10)
    f1_pheno1 = 2 * precision_pheno1 * recall_pheno1 / (precision_pheno1 + recall_pheno1)
    f1_pheno0 = 2 * precision_pheno0 * recall_pheno0 / (precision_pheno0 + recall_pheno0)
    accuracy = (cm[0,0] + cm[1,1]) / (cm[0,0] + cm[0,1] + cm[1,0] + cm[1,1])
    
    print("Recall pheno1: %f" % recall_pheno1)
    print("Recall pheno0: %f" % recall_pheno0)
    print("Precision pheno1: %f" % precision_pheno1)
    print("Precision pheno0: %f" % precision_pheno0)
    print("F1 pheno1: %f" % f1_pheno1)
    print("F1 pheno0: %f" % f1_pheno0)
    print("Accuracy: %f" % accuracy)
    print("AUC: %f" % roc_auc_score(y_test, y_pred_proba))




In [99]:
# loop through models and predict test set with best parameters

for model, names in zip(models, best_params.keys()):
    print("Predicting with model %s" % names)
    # print the model params
    model = model.set_params(**best_params[names])
    print(model)
    y_pred, y_pred_proba, cm = predict_model(model, X_train, y_train, X_test, y_test)
    print_metrics(cm, y_pred, y_pred_proba)
    print("--------------------------------------")

Predicting with model Dummy
DummyClassifier(random_state=42, strategy='most_frequent')
Recall pheno1: 1.000000
Recall pheno0: 0.000000
Precision pheno1: 0.750000
Precision pheno0: 0.000000
F1 pheno1: 0.857143
F1 pheno0: nan
Accuracy: 0.750000
AUC: 0.500000
--------------------------------------
Predicting with model SVC
SVC(C=0.1, kernel='linear', probability=True, random_state=42)
Recall pheno1: 0.928571
Recall pheno0: 0.857143
Precision pheno1: 0.951220
Precision pheno0: 0.800000
F1 pheno1: 0.939759
F1 pheno0: 0.827586
Accuracy: 0.910714
AUC: 0.982993
--------------------------------------
Predicting with model LR
LogisticRegression(C=1, max_iter=1000, random_state=42)
Recall pheno1: 0.952381
Recall pheno0: 0.857143
Precision pheno1: 0.952381
Precision pheno0: 0.857143
F1 pheno1: 0.952381
F1 pheno0: 0.857143
Accuracy: 0.928571
AUC: 0.986395
--------------------------------------
Predicting with model RF
RandomForestClassifier(n_estimators=150, random_state=42)


/var/folders/vn/mllgs9t95s722nyhhjxsd_vr0000gp/T/ipykernel_6946/4065471624.py:28: RuntimeWarning: invalid value encountered in scalar divide
  f1_pheno0 = 2 * precision_pheno0 * recall_pheno0 / (precision_pheno0 + recall_pheno0)


Recall pheno1: 1.000000
Recall pheno0: 0.928571
Precision pheno1: 0.976744
Precision pheno0: 1.000000
F1 pheno1: 0.988235
F1 pheno0: 0.962963
Accuracy: 0.982143
AUC: 1.000000
--------------------------------------
Predicting with model GB
GradientBoostingClassifier(learning_rate=0.001, n_estimators=150,
                           random_state=42)
Recall pheno1: 0.928571
Recall pheno0: 0.714286
Precision pheno1: 0.906977
Precision pheno0: 0.769231
F1 pheno1: 0.917647
F1 pheno0: 0.740741
Accuracy: 0.875000
AUC: 0.896259
--------------------------------------


In [47]:
import numpy as np
import pandas as pd

# Parameters
num_patients = 500
num_genes = 300
num_key_genes = 10  # Number of genes particularly relevant to the disease
num_pathways_per_level = [150, 50, 20, 10]
num_key_pathways = 15  # Number of pathways particularly relevant to the disease

# Simulate gene expression data
gene_expression = np.random.rand(num_patients, num_genes)

# Introduce noise
gene_expression_noise = np.random.normal(0, 0.5, gene_expression.shape)
gene_expression_noisy = gene_expression + gene_expression_noise

# Introduce correlated noise for a subset of genes
correlated_genes_indices = np.random.choice(num_genes, 200, replace=False)
correlated_noise = np.random.normal(0, 0.5, (num_patients, 1))
gene_expression_noisy[:, correlated_genes_indices] += correlated_noise

# Select key genes
key_genes = np.random.choice(num_genes, num_key_genes, replace=False)

# Create hierarchical pathways
pathways = {}
pathway_hierarchy = []
key_pathways = set()
for level in range(4):
    for i in range(num_pathways_per_level[level]):
        pathway_name = f'Level_{level}Pathway{i}'
        if level == 0:
            pathway_genes = np.random.choice(num_genes, size=np.random.randint(5, 10), replace=False)
        else:
            # Select a random parent pathway from the previous level
            parent_index = np.random.randint(num_pathways_per_level[level-1])
            parent_pathway = f'Level_{level-1}Pathway{parent_index}'
            parent_genes = pathways[parent_pathway]
            num_genes_to_choose = min(len(parent_genes), np.random.randint(3, 10))
            pathway_genes = np.random.choice(parent_genes, size=num_genes_to_choose, replace=False)
            pathway_hierarchy.append([parent_pathway, pathway_name])  # Store the parent-child relationship

        pathways[pathway_name] = pathway_genes
        if i < num_key_pathways:
            key_pathways.add(pathway_name)

# Convert pathway hierarchy to DataFrame and save
df_pathway_hierarchy = pd.DataFrame(pathway_hierarchy, columns=['Parent', 'Child'])

# Add noise pathways
num_noise_pathways = 100
for i in range(num_noise_pathways):
    pathways[f'Noise_Pathway_{i}'] = np.random.choice(num_genes, size=np.random.randint(5, 15), replace=False)

def nonlinear_transform(expression):
    # Combining multiple mathematical functions for complexity
    return np.sin(expression * np.pi) * np.tanh(expression) + np.cos(expression * np.pi / 2) * np.exp(expression)

# Function to simulate disease status with additional complexity
def simulate_disease_status(gene_expression, pathways, key_pathways):
    disease_status = []
    for patient in gene_expression:
        status = 0
        for pathway, genes in pathways.items():
            if 'Noise' not in pathway:
                mean_expression = nonlinear_transform(np.mean(patient[genes]))
                pathway_influence = 4 if pathway in key_pathways else 1
                status += pathway_influence if mean_expression > 1 else 0

        # Adjust overall threshold for disease status
        disease_threshold = len(pathways) / 3  # Adjusted to a fixed value
        disease_status.append(1 if status > disease_threshold else 0)
    return disease_status


# Generate disease status
disease_status = simulate_disease_status(gene_expression_noisy, pathways, key_pathways)

# Create DataFrame
df = pd.DataFrame(gene_expression_noisy, columns=[f'Gene_{i}' for i in range(num_genes)])
df['Disease_Status'] = disease_status

In [52]:
df_pathway_hierarchy

,Parent,Child
0,Level_0Pathway48,Level_1Pathway0
1,Level_0Pathway65,Level_1Pathway1
2,Level_0Pathway145,Level_1Pathway2
3,Level_0Pathway74,Level_1Pathway3
4,Level_0Pathway46,Level_1Pathway4
...,...,...
75,Level_2Pathway18,Level_3Pathway5
76,Level_2Pathway14,Level_3Pathway6
77,Level_2Pathway12,Level_3Pathway7
78,Level_2Pathway17,Level_3Pathway8


In [45]:
# Transpose to have patients as columns and genes as rows
df_genes = pd.DataFrame(gene_expression_noisy.T, columns=[f'Patient_{i}' for i in range(num_patients)])
df_genes.insert(0, 'Gene', [f'Gene_{i}' for i in range(num_genes)])
#df_genes.to_csv('gene_expression.csv', index=False)
df_genes.head()


,Gene,Patient_0,Patient_1,Patient_2,Patient_3,Patient_4,Patient_5,Patient_6,Patient_7,Patient_8,...,Patient_490,Patient_491,Patient_492,Patient_493,Patient_494,Patient_495,Patient_496,Patient_497,Patient_498,Patient_499
0,Gene_0,-0.431550,0.215907,2.602038,1.080208,0.532667,0.172088,0.900713,-0.101680,1.141754,...,0.464281,1.420240,-0.423890,1.469337,0.521952,0.266781,0.680056,2.100651,1.845057,1.799800
1,Gene_1,0.467136,0.729453,1.612049,-0.633879,1.377534,0.457396,0.309777,-0.354855,0.332193,...,-0.287094,1.696094,0.890454,0.425950,0.613647,-1.310581,-0.545956,0.160607,0.731972,2.255011
2,Gene_2,-0.049164,0.576522,2.400599,-0.183129,0.010567,0.123108,0.721526,0.135720,1.182385,...,1.768581,1.402315,0.654402,1.494885,1.547717,0.999256,0.536942,1.055683,1.319373,0.207473
3,Gene_3,-0.441681,1.212892,1.155304,0.007745,0.684780,-0.453528,0.316637,-0.748843,1.531549,...,1.438368,1.314571,-0.338757,1.312388,0.907144,0.763784,0.335570,0.591062,0.989662,0.381989
4,Gene_4,0.370789,-0.034772,0.509211,0.493946,0.536566,-0.458427,-0.057811,0.466342,-0.241756,...,-0.072206,0.730488,-0.030419,0.206282,-0.160211,0.701384,-0.392448,0.100521,0.142741,0.796029


In [41]:
translation_data = []
for pathway in pathways:
    if 'Level_0' in pathway:
        for gene in pathways[pathway]:
            translation_data.append([f'Gene_{gene}', pathway])

df_translation = pd.DataFrame(translation_data, columns=['Gene', 'Pathway'])
#df_translation.to_csv('gene_to_pathway.csv', index=False)
df_translation 

,Gene,Pathway
0,Gene_256,Level_0Pathway0
1,Gene_202,Level_0Pathway0
2,Gene_292,Level_0Pathway0
3,Gene_224,Level_0Pathway0
4,Gene_183,Level_0Pathway0
...,...,...
1025,Gene_211,Level_0Pathway149
1026,Gene_39,Level_0Pathway149
1027,Gene_261,Level_0Pathway149
1028,Gene_133,Level_0Pathway149


In [44]:
df_status = pd.DataFrame({'Patient': [f'Patient_{i}' for i in range(num_patients)], 'Disease_Status': disease_status})
#df_status.to_csv('patient_status.csv', index=False)
df_status.head()

,Patient,Disease_Status
0,Patient_0,1
1,Patient_1,1
2,Patient_2,0
3,Patient_3,1
4,Patient_4,1


In [46]:
# Create a parent-child mapping for pathways
pathway_hierarchy = []
for level in range(1, 4):  # Start from level 1 as level 0 has no parent
    for i in range(num_pathways_per_level[level]):
        child_pathway = f'Level_{level}Pathway{i}'
        # Find the parent pathway
        parent_pathway = next((p for p in pathways if child_pathway in pathways[p]), None)
        if parent_pathway:
            pathway_hierarchy.append([parent_pathway, child_pathway])

df_pathway_hierarchy = pd.DataFrame(pathway_hierarchy, columns=['Parent', 'Child'])
#df_pathway_hierarchy.to_csv('pathway_hierarchy.csv', index=False)
df_pathway_hierarchy.head()

/tmp/ipykernel_267856/3613225667.py:7: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  parent_pathway = next((p for p in pathways if child_pathway in pathways[p]), None)


,Parent,Child


In [33]:
# split data into training and test set

X_train, X_test, y_train, y_test = train_test_split(df.drop('Disease_Status', axis=1), df['Disease_Status'], test_size=0.2, random_state=42)

In [34]:
# train log reg model
log_reg = LogisticRegression(random_state=42, max_iter=1000)
log_reg.fit(X_train, y_train)

# predict test set
y_pred = log_reg.predict(X_test)

# calculate acc
acc = (y_pred == y_test).mean()
print(f'Accuracy: {acc}')

Accuracy: 0.86
